Here we perform a query on a database like sum before but return whether the sum is above or below a certain threshold.

The steps for this project are:

1) Create the above described query function

2) Create 10 databases of size 10 and query them with a threshold of 5

3) Print out the sensitivity of all the 10 databases

In [1]:
import torch

def th_query(db, threshold=5):
    return (db.sum() > threshold).float()

In [4]:
# first lets write our database generation function

def get_parallel_db(db,remove_index):
    return torch.cat((db[0:remove_index], db[remove_index+1:]))

def get_parallel_dbs(db):
    parallel_dbs = list()
    
    for i in range(len(db)):
        parallel_dbs.append(get_parallel_db(db, i))
    
    return parallel_dbs

def create_db_and_pdbs(num_entries):
    
    db = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(db)
    
    return db, pdbs

In [22]:
db, pdbs = create_db_and_pdbs(10)
th_query(db) #sometimes we get 1(True) and at others 0(False) as we rerun the cell

tensor(1.)

In [23]:
# lets take a db with sum 6
db.sum()

tensor(6)

In [36]:
# Now the query returns true and hence some pdbs will give 0 output for the query as one person is removed from them
# However with only 4 as sum we will have no sensitivity at all
# Now we can evaluate the sensitivity for 10 ddbs andtheir pdbs using the sensitivity function

def sensitivity(query, num_entries):
    db = torch.rand(num_entries)>0.5
    pdbs = list()
    for i in range(len(db)):
        pdbs.append(torch.cat((db[0:i], db[i+1:])))
    full_db_result = query(db)
    max_distance = 0
    for pdb in pdbs:
        pdb_result = query(pdb)
        db_distance = torch.abs(pdb_result - full_db_result)
        if db_distance > max_distance:
            max_distance = db_distance
    return max_distance

for i in range(10):
    sens_f = sensitivity(th_query, num_entries=10)
    print(sens_f)

0
0
0
0
0
0
0
0
tensor(1.)
tensor(1.)


In [ ]:
# Here our new threshold query has a variable sensitivity because when a database has sum 6 we get a maximum 
# sensitivity of 1 and if the sum of the database is below 6 then its definitely a 0 regaardess of who we 
# remove. Same is the case when we have a sum greater than 6 then sensitvity will always be 1.